In [1]:
import json
import prompt_rag
import torch
import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

dirs = ["../..", ".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

import covmis

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType

# model_type = CustomModelType.mixtral_moe_7b_instruct_awq
# model_type = CustomModelType.llama_3_70b_instruct_awq
model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=1,
    max_model_len=4096,
    # gpu_memory_utilization=0.95,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True, 
)

search_engine = "brave"
model_name = 'solar'
K = 5
sort = False

data_search = covmis.load_train_search()
data_train = covmis.load_train()

# data_search = data_search[:10] + [data_search[9690]]
prompt_rag.update_train_search_llm(
    model_name, get_resp_list, search_engine,
    data_train, data_search, K=K, sort=sort
)


2024-06-28 23:29:58,788 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-06-28 23:29:58,791 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-06-28 23:29:58,826 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 21bc0b9ccf26bcf3f4ca2e675ec8875d and a total number of 980 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  

WARNING 06-28 23:29:59 config.py:217] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-06-28 23:30:01,808	INFO worker.py:1753 -- Started a local Ray instance.


INFO 06-28 23:30:02 config.py:623] Defaulting to use mp for distributed inference
INFO 06-28 23:30:02 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ', speculative_config=None, tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42, served_model_name=/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ)
INFO 06-28 23:30:02 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-28 23:30:02 selector.py:51] Usin

(raylet) [2024-06-28 23:30:11,769 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16864354304; capacity: 1967317549056. Object creation will fail if spilling is required.


...

WARNING 06-29 02:17:52 scheduler.py:1089] Sequence group 8065 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


100%|█████████▉| 14380/14384 [4:55:08<00:05,  1.47s/it](raylet) [2024-06-29 04:26:35,946 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16607363072; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:26:45,959 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16607354880; capacity: 1967317549056. Object creation will fail if spilling is required.
100%|██████████| 14384/14384 [4:55:25<00:00,  1.23s/it]
(raylet) [2024-06-29 04:26:55,975 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16607350784; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:27:05,987 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /

(raylet) [2024-06-29 04:27:25,999 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16578301952; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:27:36,009 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16578297856; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:27:46,020 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16578285568; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:27:56,031 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16578273280; capacity: 19

(VllmWorkerProcess pid=3764957) ERROR 06-29 04:56:51 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method start_worker_execution_loop: [../third_party/gloo/gloo/transport/tcp/unbound_buffer.cc:81] Timed out waiting 1800000ms for recv operation to complete, Traceback (most recent call last):
(VllmWorkerProcess pid=3764957) ERROR 06-29 04:56:51 multiproc_worker_utils.py:226]   File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/vllm/executor/multiproc_worker_utils.py", line 223, in _run_worker_process
(VllmWorkerProcess pid=3764957) ERROR 06-29 04:56:51 multiproc_worker_utils.py:226]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=3764957) ERROR 06-29 04:56:51 multiproc_worker_utils.py:226]              ^^^^^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=3764957) ERROR 06-29 04:56:51 multiproc_worker_utils.py:226]   File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 115

(raylet) [2024-06-29 04:56:58,089 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16438644736; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:57:08,101 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16438181888; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:57:18,113 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16438169600; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-06-29 04:57:28,126 E 3761357 3761390] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-28_23-29-59_539519_3759991 is over 95% full, available space: 16438169600; capacity: 19